# Optimisation for Deep Learning

Learning outcomes
- understand mathematically and intuitively the most common optimisation algorithms used for optimising deep models
- implement your own optimiser in PyTorch



## Gradient descent

## SGD

## AdaGrad



## RMSProp



## Adam

## So which algorithm do I use?

Well... as usual, it depends on your problem and your dataset.

It's still a highly active field of research. But in general, **SGD with momentum or Adam** are the go to choices for optimising deep models.

## Using these optimisation algorithms

Let's set up the same neural network as in the previous module, and then switch out the optimiser for Adam and others and show how you can adapt it to use momentum.

In [29]:
import sys
sys.path.append('..')
from utils import NN, get_dataloaders
import torch
import torch.nn.functional as F

my_nn = NN([784, 1024, 1024, 512, 10], distribution=True, flatten_input=True)

learning_rate = 0.001

# HOW TO USE DIFFERENT OPTIMISERS PROVIDED BY PYTORCH
optimiser = torch.optim.SGD(my_nn.parameters(), lr=learning_rate, momentum=0.1)
# optimiser = torch.optim.Adagrad(NN.parameters(), lr=learning_rate)
# optimiser = torch.optim.RMSprop(NN.parameters(), lr=learning_rate)
optimiser = torch.optim.Adam(my_nn.parameters(), lr=learning_rate)

The stuff below is exactly the same as before!

In [30]:
# GET DATALOADERS
test_loader, val_loader, train_loader = get_dataloaders()
criterion = F.cross_entropy

# SET UP TRAINING VISUALISATION
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir='../../runs')                            # we will use this to show our models performance on a graph

# TRAINING LOOP
def train(model, optimiser, epochs=1, tag='Loss/Train'):
    for epoch in range(epochs):
        for idx, minibatch in enumerate(train_loader):
            inputs, labels = minibatch
            prediction = model(inputs)             # pass the data forward through the model
            loss = criterion(prediction, labels)   # compute the loss
            print('Epoch:', epoch, '\tBatch:', idx, '\tLoss:', loss)
            optimiser.zero_grad()                  # reset the gradients attribute of each of the model's params to zero
            loss.backward()                        # backward pass to compute and set all of the model param's gradients
            optimiser.step()                       # update the model's parameters
            writer.add_scalar(tag, loss, epoch*len(train_loader) + idx)    # write loss to a graph

train(my_nn, optimiser)

 	Loss: tensor(1.9378, grad_fn=<NllLossBackward>)
Epoch: 0 	Batch: 340 	Loss: tensor(1.5862, grad_fn=<NllLossBackward>)
Epoch: 0 	Batch: 341 	Loss: tensor(1.6386, grad_fn=<NllLossBackward>)
Epoch: 0 	Batch: 342 	Loss: tensor(1.8936, grad_fn=<NllLossBackward>)
Epoch: 0 	Batch: 343 	Loss: tensor(1.8180, grad_fn=<NllLossBackward>)
Epoch: 0 	Batch: 344 	Loss: tensor(1.9209, grad_fn=<NllLossBackward>)
Epoch: 0 	Batch: 345 	Loss: tensor(1.7717, grad_fn=<NllLossBackward>)
Epoch: 0 	Batch: 346 	Loss: tensor(1.7141, grad_fn=<NllLossBackward>)
Epoch: 0 	Batch: 347 	Loss: tensor(1.6657, grad_fn=<NllLossBackward>)
Epoch: 0 	Batch: 348 	Loss: tensor(1.5850, grad_fn=<NllLossBackward>)
Epoch: 0 	Batch: 349 	Loss: tensor(1.6648, grad_fn=<NllLossBackward>)
Epoch: 0 	Batch: 350 	Loss: tensor(1.6003, grad_fn=<NllLossBackward>)
Epoch: 0 	Batch: 351 	Loss: tensor(1.8465, grad_fn=<NllLossBackward>)
Epoch: 0 	Batch: 352 	Loss: tensor(1.6780, grad_fn=<NllLossBackward>)
Epoch: 0 	Batch: 353 	Loss: tensor(1.775

Let's compare the training curves generated using some of the optimisers that we explained above.

In [27]:
optimisers = [
    {
        'optimiser_class': torch.optim.SGD, 
        'tag': 'SGD'
    },
    {
        'optimiser_class': torch.optim.Adam,
        'tag': 'Adam'
    },
    {
        'optimiser_class': torch.optim.Adagrad,
        'tag': 'Adagrad'
    },
    {
        'optimiser_class': torch.optim.RMSprop,
        'tag': 'RMSProp'
    }
]

for optimiser_obj in optimisers:    
    my_nn = NN([784, 1024, 1024, 512, 10], distribution=True, flatten_input=True)
    optimiser_class = optimiser_obj['optimiser_class']
    optimiser = optimiser_class(my_nn.parameters(), lr=0.01)
    tag = optimiser_obj['tag']
    train(my_nn, optimiser, epochs=1, tag=f'Loss/Train/{tag}')
    

Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/ice/.local/lib/python3.8/site-packages/tensorboard/summary/writer/event_file_writer.py", line 238, in run
    self._record_writer.write(data)
  File "/home/ice/.local/lib/python3.8/site-packages/tensorboard/summary/writer/record_writer.py", line 40, in write
    self._writer.write(header + header_crc + data + footer_crc)
  File "/home/ice/.local/lib/python3.8/site-packages/tensorboard/compat/tensorflow_stub/io/gfile.py", line 535, in write
    self.fs.append(self.filename, file_content, self.binary_mode)
  File "/home/ice/.local/lib/python3.8/site-packages/tensorboard/compat/tensorflow_stub/io/gfile.py", line 158, in append
    self._write(filename, file_content, "ab" if binary_mode else "a")
  File "/home/ice/.local/lib/python3.8/site-packages/tensorboard/compat/tensorflow_stub/io/gfile.py", line 162, in _write
    with 

KeyboardInterrupt: 

## Implementing our own PyTorch optimiser

To understand a bit further what's happening under the hood, let's implement SGD from scratch.

In [ ]:
class SGD():
    def __init__(self, model_params, learning_rate):
        self.model_params = model_params
        self.learning_rate

    def step(self):
        for param in self.model_params:
            param -= self.learning_rate * param.grad

    def zero_grad(self):
        for param in self.model_params:
            param.grad = torch.zeros_like(param.grad)

In [ ]:
my_nn = NN()
optimiser = SGD()

train(my_nn, optimiser)